In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/santander-product-recommendation/test_ver2.csv.zip
/kaggle/input/santander-product-recommendation/sample_submission.csv.zip
/kaggle/input/santander-product-recommendation/train_ver2.csv.zip


In [2]:
import gc
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

# 加载数据

In [4]:
train = pd.read_csv('/kaggle/input/santander-product-recommendation/train_ver2.csv.zip')
test = pd.read_csv('/kaggle/input/santander-product-recommendation/test_ver2.csv.zip')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 数据预处理
## 数据清洗

In [5]:
# train.loc[train['age'].astype('str').str.contains('NA'),'age'] = np.nan
train['age'] = pd.to_numeric(train['age'], errors='coerce')
train['antiguedad'] = pd.to_numeric(train['antiguedad'], errors='coerce')
train['renta'] = pd.to_numeric(train['renta'], errors='coerce')
train.loc[train['antiguedad']<0,'antiguedad'] = 0

test['age'] = pd.to_numeric(test['age'], errors='coerce')
test['antiguedad'] = pd.to_numeric(test['antiguedad'], errors='coerce')
test['renta'] = pd.to_numeric(test['renta'], errors='coerce')
test.loc[test['antiguedad']<0,'antiguedad'] = 0

train['indrel_1mes'] = train['indrel_1mes'].map({1.0:'1', '1.0':'1', '1':'1', 
                           2.0:'2', '2.0':'2', '2':'2', 
                          3.0:'3', '3.0':'3','3':'3', 
                          4.0:'4','4.0':'4', '4':'4',
                         'P':'P'})
test['indrel_1mes'] = test['indrel_1mes'].map({1.0:'1', '1.0':'1', '1':'1', 
                           2.0:'2', '2.0':'2', '2':'2', 
                          3.0:'3', '3.0':'3','3':'3', 
                          4.0:'4','4.0':'4', '4':'4',
                         'P':'P'})

In [7]:
string_data = train.select_dtypes(include=["object"])
missing_columns = [col for col in string_data if string_data[col].isnull().any()]
# for col in missing_columns:
#     print("Unique values for {0}:\n{1}\n".format(col,string_data[col].unique()))
del string_data

## 填充缺失值

In [8]:
train['indfall'] = train['indfall'].fillna('N')
train['tiprel_1mes'] = train['tiprel_1mes'].fillna('A')
train['indrel_1mes'] = train['indrel_1mes'].fillna('P')

test['indfall'] = test['indfall'].fillna('N')
test['tiprel_1mes'] = test['tiprel_1mes'].fillna('A')
test['indrel_1mes'] = test['indrel_1mes'].fillna('P')

unknown_cols = [col for col in missing_columns if col not in ["indfall","tiprel_1mes","indrel_1mes"]]

for column in unknown_cols:
    train[column] = train[column].fillna('UNKNOWN')
    test[column] = test[column].fillna('UNKNOWN')

## 分类特征LabelEncode

In [11]:
cat_columns = ['ind_empleado', 'pais_residencia', 'sexo', 'indrel_1mes', 'tiprel_1mes',
               'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'segmento']

encoders = []
for i in cat_columns:
    le = LabelEncoder()
    train[i] = le.fit_transform(train[i])
    test[i] = le.transform(test[i])
    encoders.append(le)

In [13]:
features = ['ind_empleado', 'pais_residencia', 'sexo',
            'age', 'ind_nuevo', 'antiguedad', 'indrel',
             'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
            'conyuemp', 'canal_entrada', 'indfall', 'cod_prov',
            'ind_actividad_cliente', 'renta', 'segmento']

## 处理label

In [14]:
targets = train.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
targets = targets[2:]

for column in targets:
    train[column] = train[column].fillna(0)

## 使用2015-06-28数据进行训练，2015-01-28，2015-05-28数据构造新特征

In [16]:
train = train[(train['fecha_dato']=='2015-01-28')|(train['fecha_dato']=='2015-05-28')|(train['fecha_dato']=='2015-06-28')]

lag_cust = train[(train['fecha_dato']=='2015-01-28')][['ncodpers']+list(targets)]
cust = train[(train['fecha_dato']=='2015-05-28')][['ncodpers']+list(targets)]
lag_cust2 = train[(train['fecha_dato']=='2016-01-28')][['ncodpers']+list(targets)]
cust2 = train[(train['fecha_dato']=='2016-05-28')][['ncodpers']+list(targets)]

In [18]:
train_data = train[train['fecha_dato']=='2015-06-28'][['ncodpers']+features].merge(cust,on='ncodpers',how='left').merge(lag_cust,on='ncodpers',how='left')
test_data = test[['ncodpers']+features].merge(cust2,on='ncodpers',how='left').merge(lag_cust2,on='ncodpers',how='left')

In [19]:
for i in targets:
    train_data[i+'_x'] = train_data[i+'_x'].fillna(0)
    train_data[i+'_y'] = train_data[i+'_y'].fillna(0)
    test_data[i+'_x'] = test_data[i+'_x'].fillna(0)
    test_data[i+'_y'] = test_data[i+'_y'].fillna(0)

In [20]:
target_df = train[train['fecha_dato'] == '2015-06-28'][['ncodpers']+list(targets)].merge(train[train['fecha_dato'] == '2015-05-28'][['ncodpers']+list(targets)],
                                                                                         on='ncodpers', how='left')

In [21]:
target_df_1 = target_df[['ncodpers']+[i+'_x' for i in targets]]
target_df_1.columns = ['ncodpers'] + list(targets)
target_df_2 = target_df[['ncodpers']+[i+'_y' for i in targets]]
target_df_2.columns = ['ncodpers'] + list(targets)
target_df_1.set_index('ncodpers',inplace=True)
target_df_2.set_index('ncodpers',inplace=True)

target_df = (target_df_1 - target_df_2).reset_index()
target_df[target_df<0]=0

In [23]:
for i in targets:
    print(i,target_df[i].sum())

ind_cco_fin_ult1 6588.0
ind_cder_fin_ult1 9.0
ind_cno_fin_ult1 1857.0
ind_ctju_fin_ult1 7.0
ind_ctma_fin_ult1 210.0
ind_ctop_fin_ult1 213.0
ind_ctpp_fin_ult1 151.0
ind_deco_fin_ult1 289.0
ind_deme_fin_ult1 33.0
ind_dela_fin_ult1 939.0
ind_ecue_fin_ult1 1204.0
ind_fond_fin_ult1 245.0
ind_hip_fin_ult1 4.0
ind_plan_fin_ult1 19.0
ind_pres_fin_ult1 8.0
ind_reca_fin_ult1 2931.0
ind_tjcr_fin_ult1 4655.0
ind_valo_fin_ult1 152.0
ind_viv_fin_ult1 3.0
ind_nomina_ult1 5070.0
ind_nom_pens_ult1 8135.0
ind_recibo_ult1 9023.0


In [24]:
columns = train_data.columns
train_data = train_data.merge(target_df,on='ncodpers')

data_list = []
for i,column in enumerate(targets):
    data = train_data[train_data[column]>0][columns]
    data['target'] = i
    data_list.append(data)
train_data = pd.concat(data_list)

# 模型训练

In [26]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import xgboost as xgb

## lightgbm

In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data.drop(columns=['ncodpers','target']), train_data['target'], test_size=0.2, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

In [28]:
param = {'boosting_type':'gbdt',
         'objective' : 'multiclass',
         'num_class':22,
         'metric' : 'multi_logloss',
         'learning_rate' : 0.01,
         'max_depth' : 15,
         'feature_fraction':0.8,
         'bagging_fraction': 0.9,
         'bagging_freq': 8,
         'lambda_l1': 0.6,
         'lambda_l2': 0,
}

In [29]:
model = lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round = 10000 ,early_stopping_rounds=200,verbose_eval=25, categorical_feature=cat_columns)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['canal_entrada', 'conyuemp', 'ind_empleado', 'indext', 'indfall', 'indrel_1mes', 'indresi', 'pais_residencia', 'segmento', 'sexo', 'tiprel_1mes']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds
[25]	training's multi_logloss: 1.83789	valid_1's multi_logloss: 1.85691
[50]	training's multi_logloss: 1.66053	valid_1's multi_logloss: 1.69021
[75]	training's multi_logloss: 1.53281	valid_1's multi_logloss: 1.57058
[100]	training's multi_logloss: 1.43802	valid_1's multi_logloss: 1.48248
[125]	training's multi_logloss: 1.36348	valid_1's multi_logloss: 1.41441
[150]	training's multi_logloss: 1.30284	valid_1's multi_logloss: 1.36008
[175]	training's multi_logloss: 1.25432	valid_1's multi_logloss: 1.31784
[200]	training's multi_logloss: 1.21389	valid_1's multi_logloss: 1.28321
[225]	training's multi_logloss: 1.17998	valid_1's multi_logloss: 1.25527
[250]	training's multi_logloss: 1.15083	valid_1's multi_logloss: 1.23191
[275]	training's multi_logloss: 1.12539	valid_1's multi_logloss: 1.21217
[300]	training's multi_logloss: 1.10392	valid_1's multi_logloss: 1.19648
[325]	training's multi_logloss: 1.08518	valid_1's multi_logloss: 

## 预测结果

In [33]:
preds=model.predict(test_data.drop(columns=['ncodpers']))
preds_idx = np.argsort(-preds, axis=1)[:,:7]

test_id = test['ncodpers']
products = [' '.join(targets[i]) for i in preds_idx]

df = pd.DataFrame({'ncodpers':test_id, 'added_products':products})
df.to_csv('lgb.csv', index=False)